In [1]:
# Import libraries and Dependencies to be used in the project
import pandas as pd
import numpy as np
import requests
import json
import os
from dotenv import load_dotenv

In [2]:
#get private environment varibles from .env file
load_dotenv()
client_id = os.environ['TWITCH_CLIENT_ID']
client_secret = os.environ['TWITCH_CLIENT_SECRET']

In [3]:
#Define the Twitch varibles for a request to the twitch API to get the access token

twitch_url = 'https://id.twitch.tv/oauth2/token'
params = {
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'client_credentials'
}

In [4]:
#make request to twitch API to get access token and store it in a variable
twitch_request = requests.post(twitch_url, data=params)

In [5]:
#Print the response from the request from Twitch API
print(twitch_request.text)


{"access_token":"e7s5t4ogdibtg86s2jnjvm1tq1xy7r","expires_in":5424735,"token_type":"bearer"}



In [6]:
# Print the response from the request from Twitch API in a more readable format using json
print(json.dumps(twitch_request.json(), indent=4))

{
    "access_token": "e7s5t4ogdibtg86s2jnjvm1tq1xy7r",
    "expires_in": 5424735,
    "token_type": "bearer"
}


In [7]:
#Save the access token so it can be used in next IGDB API call
bearer_token = twitch_request.json()['access_token']

In [8]:
# Define the IGDB varibles for a request to the IGDB API to get the access token
IGDB_URL = 'https://api.igdb.com/v4/games'

In [9]:
#Function to make API request to IGDB
def make_api_request(base_url, df_name):   
   offset = 0
   all_results = []
   #loop through all the results, 500 at a time since that is the max limit
   while True:
      
      #make request to IGDB API
      IGDB_request = requests.post(base_url, headers={'Client-ID': client_id, 'Authorization': f'Bearer {bearer_token }'}, data=f'fields *; limit 500; offset {offset};')
      IGDB_data = IGDB_request.json()
      
      all_results.extend(IGDB_data)
      
      #if it pulls less than 500 results, then it has reached the end of the data, so break the loop
      if len(IGDB_data) < 500:
         break
      
      offset += len(IGDB_data)
   #Put the results into a pandas normalized dataframe
   df_name = pd.json_normalize(all_results)
   df_name.head()
   return df_name

In [10]:
#Make API request to IGDB to get the data for the video games
VG_DF = make_api_request(IGDB_URL, 'VG_DF')

KeyboardInterrupt: 

In [ ]:
# Print the column information for the dataframe
VG_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273760 entries, 0 to 273759
Data columns (total 58 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       273760 non-null  int64  
 1   age_ratings              61352 non-null   object 
 2   alternative_names        61299 non-null   object 
 3   category                 273760 non-null  int64  
 4   cover                    209374 non-null  float64
 5   created_at               273760 non-null  int64  
 6   external_games           255072 non-null  object 
 7   first_release_date       175910 non-null  float64
 8   game_modes               153146 non-null  object 
 9   genres                   223225 non-null  object 
 10  involved_companies       119349 non-null  object 
 11  keywords                 90719 non-null   object 
 12  name                     273760 non-null  object 
 13  platforms                187136 non-null  object 
 14  play

In [ ]:
# Print and list the column names for the dataframe
VG_DF.columns

Index(['id', 'age_ratings', 'alternative_names', 'category', 'cover',
       'created_at', 'external_games', 'first_release_date', 'game_modes',
       'genres', 'involved_companies', 'keywords', 'name', 'platforms',
       'player_perspectives', 'release_dates', 'screenshots', 'similar_games',
       'slug', 'storyline', 'summary', 'tags', 'themes', 'updated_at', 'url',
       'videos', 'websites', 'checksum', 'game_localizations', 'collection',
       'multiplayer_modes', 'status', 'language_supports', 'collections',
       'hypes', 'artworks', 'version_parent', 'version_title', 'parent_game',
       'bundles', 'franchises', 'follows', 'rating', 'rating_count',
       'total_rating', 'total_rating_count', 'game_engines',
       'aggregated_rating', 'aggregated_rating_count', 'ports', 'dlcs',
       'remakes', 'franchise', 'expansions', 'standalone_expansions', 'forks',
       'remasters', 'expanded_games'],
      dtype='object')

In [ ]:
#Testing to see if the data was pulled correctly
BG3 = VG_DF.loc[VG_DF['name'] == "Baldur's Gate 3"]
BG3

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,aggregated_rating_count,ports,dlcs,remakes,franchise,expansions,standalone_expansions,forks,remasters,expanded_games
86294,119171,"[148088, 161963, 162025, 162026, 162027, 162028]","[50217, 78595, 78596, 78597, 139162]",0,289025.0,1559228938,"[1725312, 1775824, 1914473, 2070558, 2678929, ...",1.601942e+09,"[1, 2, 3, 4]","[12, 15, 16, 24, 31]",...,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Testing to make sure the .datetime works on this data (this took forever)
BG3['first_release_date'] = pd.to_datetime(BG3['first_release_date'], infer_datetime_format=True, unit='s')


C:\Users\Keegan\AppData\Local\Temp\ipykernel_16756\3513838881.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  BG3['first_release_date'] = pd.to_datetime(BG3['first_release_date'], infer_datetime_format=True, unit='s')
C:\Users\Keegan\AppData\Local\Temp\ipykernel_16756\3513838881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BG3['first_release_date'] = pd.to_datetime(BG3['first_release_date'], infer_datetime_format=True, unit='s')


In [ ]:
# List the dataframe for Baldurs Gate 3
BG3

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,aggregated_rating_count,ports,dlcs,remakes,franchise,expansions,standalone_expansions,forks,remasters,expanded_games
86294,119171,"[148088, 161963, 162025, 162026, 162027, 162028]","[50217, 78595, 78596, 78597, 139162]",0,289025.0,1559228938,"[1725312, 1775824, 1914473, 2070558, 2678929, ...",2020-10-06,"[1, 2, 3, 4]","[12, 15, 16, 24, 31]",...,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#make a copy of the dataframe so that the original data is not altered
copy_df = VG_DF.copy()  

In [ ]:
#drop any rows that have a NaN value in the 'first_release_date' column
copy_df = copy_df.dropna(subset=['first_release_date'], how='any')

copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175910 entries, 0 to 273759
Data columns (total 58 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       175910 non-null  int64  
 1   age_ratings              54829 non-null   object 
 2   alternative_names        51028 non-null   object 
 3   category                 175910 non-null  int64  
 4   cover                    156219 non-null  float64
 5   created_at               175910 non-null  int64  
 6   external_games           159889 non-null  object 
 7   first_release_date       175910 non-null  float64
 8   game_modes               127186 non-null  object 
 9   genres                   157802 non-null  object 
 10  involved_companies       109430 non-null  object 
 11  keywords                 76723 non-null   object 
 12  name                     175910 non-null  object 
 13  platforms                175910 non-null  object 
 14  player_pe

In [ ]:
#convert the 'first_release_date' column to a datetime object
copy_df['first_release_date'] = pd.to_datetime(copy_df['first_release_date'], errors = 'coerce',  unit='s')


In [ ]:
# Print and generate a copy of the dataframe
copy_df.head()

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,aggregated_rating_count,ports,dlcs,remakes,franchise,expansions,standalone_expansions,forks,remasters,expanded_games
0,131913,[101730],"[40085, 40086, 40087]",0,267633.0,1584788069,[1977891],2016-09-21,[1],[34],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88308,NaN,NaN,0,64849.0,1519237439,"[243459, 1960113]",2016-11-09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63308,[14739],NaN,0,78658.0,1505087910,"[12596, 118080, 1185144]",2013-07-03,"[2, 3]",[5],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,231577,"[114607, 114609, 126972, 126973, 126974, 127011]",NaN,0,280467.0,1673878007,"[2639959, 2677882, 2677897, 2677919, 2678595]",2023-02-23,"[1, 2]",[15],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,119025,NaN,NaN,0,NaN,1558701524,"[1723803, 1957721]",2019-06-08,[1],[13],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Define a string variable for the IGDB API request to get the release dates
release_dates_url = 'https://api.igdb.com/v4/release_dates'


In [ ]:
#make API request to IGDB to get the data for the release dates
RD_DF = make_api_request(release_dates_url, 'RD_DF')
RD_DF.head()

,id,category,created_at,date,game,human,m,platform,region,updated_at,y,checksum,status
0,298,2,1301923501,7.258464e+08,110,1993,1.0,6,8,1339423931,1993.0,09293025-b7e0-1565-8848-532914a16081,NaN
1,165447,0,1550867412,1.561421e+09,115477,"Jun 25, 2019",6.0,6,8,1550867489,2019.0,79b1fc63-2f73-5351-c0f2-6b4fe6cb33fb,NaN
2,143501,0,1517637179,1.210118e+09,37605,"May 07, 2008",5.0,5,5,1517864621,2008.0,5114450a-b7e1-c105-a3b9-08ad7b2e4321,NaN
3,240506,0,1619125310,1.622160e+09,56914,"May 28, 2021",5.0,48,2,1619163937,2021.0,d4c4a16e-f78d-ae3d-9922-04b3b5a7cdad,NaN
4,310513,0,1631498490,6.318432e+08,48393,"Jan 09, 1990",1.0,99,5,1631499375,1990.0,530d65c3-cbbe-9a9a-eea0-a999c0e5c61f,NaN


In [ ]:
#convert the 'date' column to a datetime object
RD_DF['date'] = pd.to_datetime(RD_DF['date'], errors = 'coerce', unit='s')
RD_DF.head()

,id,category,created_at,date,game,human,m,platform,region,updated_at,y,checksum,status
0,298,2,1301923501,1993-01-01,110,1993,1.0,6,8,1339423931,1993.0,09293025-b7e0-1565-8848-532914a16081,NaN
1,165447,0,1550867412,2019-06-25,115477,"Jun 25, 2019",6.0,6,8,1550867489,2019.0,79b1fc63-2f73-5351-c0f2-6b4fe6cb33fb,NaN
2,143501,0,1517637179,2008-05-07,37605,"May 07, 2008",5.0,5,5,1517864621,2008.0,5114450a-b7e1-c105-a3b9-08ad7b2e4321,NaN
3,240506,0,1619125310,2021-05-28,56914,"May 28, 2021",5.0,48,2,1619163937,2021.0,d4c4a16e-f78d-ae3d-9922-04b3b5a7cdad,NaN
4,310513,0,1631498490,1990-01-09,48393,"Jan 09, 1990",1.0,99,5,1631499375,1990.0,530d65c3-cbbe-9a9a-eea0-a999c0e5c61f,NaN


In [ ]:
#remove unnecessary columns
dates_ids = RD_DF[['id', 'date']]
dates_ids.head()

,id,date
0,298,1993-01-01
1,165447,2019-06-25
2,143501,2008-05-07
3,240506,2021-05-28
4,310513,1990-01-09


In [ ]:
#rename columns
dates_ids = dates_ids.rename(columns={'id': 'release_date_id', 'date':'release_date_data' })
dates_ids.head()

,release_date_id,release_date_data
0,298,1993-01-01
1,165447,2019-06-25
2,143501,2008-05-07
3,240506,2021-05-28
4,310513,1990-01-09


In [ ]:
#merge the two dataframes, 'copy_df' and 'dates_ids', on the 'release_dates' column, which is a list of release date ids
merged_df = copy_df.explode('release_dates').merge(dates_ids, left_on='release_dates', right_on='release_date_id', how='left')
merged_df[['name', 'first_release_date', 'release_date_data', 'release_dates', 'release_date_id']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346465 entries, 0 to 346464
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   name                346465 non-null  object        
 1   first_release_date  346464 non-null  datetime64[ns]
 2   release_date_data   342352 non-null  datetime64[ns]
 3   release_dates       346465 non-null  object        
 4   release_date_id     346465 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 13.2+ MB


In [ ]:
#drop the 'release_dates' column
merged_df['release_dates'] = merged_df['release_date_data']
merged_df = merged_df.drop(columns=['release_date_data', 'release_date_id'])


In [ ]:
merged_df[['name', 'release_dates']].head()

,name,release_dates
0,Maji Kyun! Renaissance,2016-09-21
1,Hey Duggee: The Big Outdoor App,2016-11-09
2,Soldier Front 2,2013-07-03
3,Blood Bowl 3: Black Orcs Edition,2023-02-23
4,Blood Bowl 3: Black Orcs Edition,2023-02-23


In [ ]:
#Testing to see if the data was merged correctly, it was NOT, these should be one row, i coulnd't figure out how to fix it, so i just dropped the duplicates
##COME BACK TO THIS
BG3_2 = merged_df.loc[merged_df['name'] == "Baldur's Gate 3"]
BG3_2[['release_dates', 'first_release_date', 'name']].head()

,release_dates,first_release_date,name
77660,2023-09-06,2020-10-06,Baldur's Gate 3
77661,2023-08-03,2020-10-06,Baldur's Gate 3
77662,2023-12-08,2020-10-06,Baldur's Gate 3
77663,2020-10-06,2020-10-06,Baldur's Gate 3
77664,2020-10-06,2020-10-06,Baldur's Gate 3


In [ ]:
#output the data to a parquet file, (instead of csv) since it is a large file and github does not allow files over 100MB
merged_df.to_csv('uncleaned_video_games.csv', index=False)

In [ ]:
#read the data back in to make sure it was saved correctly
Video_games_df = pd.read_csv('uncleaned_video_games.csv')
Video_games_df.columns

Index(['id', 'age_ratings', 'alternative_names', 'category', 'cover',
       'created_at', 'external_games', 'first_release_date', 'game_modes',
       'genres', 'involved_companies', 'keywords', 'name', 'platforms',
       'player_perspectives', 'release_dates', 'screenshots', 'similar_games',
       'slug', 'storyline', 'summary', 'tags', 'themes', 'updated_at', 'url',
       'videos', 'websites', 'checksum', 'game_localizations', 'collection',
       'multiplayer_modes', 'status', 'language_supports', 'collections',
       'hypes', 'artworks', 'version_parent', 'version_title', 'parent_game',
       'bundles', 'franchises', 'follows', 'rating', 'rating_count',
       'total_rating', 'total_rating_count', 'game_engines',
       'aggregated_rating', 'aggregated_rating_count', 'ports', 'dlcs',
       'remakes', 'franchise', 'expansions', 'standalone_expansions', 'forks',
       'remasters', 'expanded_games'],
      dtype='object')

In [ ]:
#drop unnecessary columns from the dataframe that are not needed for the analysis
cols_to_delete = ['age_ratings', 'alternative_names','cover','created_at', 'external_games','game_modes','involved_companies', 'keywords','platforms',
       'player_perspectives','screenshots', 'similar_games',
       'slug', 'storyline', 'summary', 'tags', 'themes', 'updated_at', 'url',
       'videos', 'websites', 'checksum', 'game_localizations', 'collection',
       'multiplayer_modes', 'status', 'language_supports', 'collections',
       'hypes', 'artworks', 'version_parent', 'version_title', 'parent_game',
       'bundles', 'franchises', 'follows','game_engines','ports', 'dlcs',
       'remakes', 'franchise', 'expansions', 'standalone_expansions', 'forks',
       'remasters', 'expanded_games', 'genres']
cleaned_df = Video_games_df.drop(columns=cols_to_delete)
cleaned_df.head()

,id,category,first_release_date,name,release_dates,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
0,131913,0,2016-09-21,Maji Kyun! Renaissance,2016-09-21,NaN,NaN,NaN,NaN,NaN,NaN
1,88308,0,2016-11-09,Hey Duggee: The Big Outdoor App,2016-11-09,NaN,NaN,NaN,NaN,NaN,NaN
2,63308,0,2013-07-03,Soldier Front 2,2013-07-03,NaN,NaN,NaN,NaN,NaN,NaN
3,231577,0,2023-02-23,Blood Bowl 3: Black Orcs Edition,2023-02-23,NaN,NaN,NaN,NaN,NaN,NaN
4,231577,0,2023-02-23,Blood Bowl 3: Black Orcs Edition,2023-02-23,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Print the column information for the dataframe
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346465 entries, 0 to 346464
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   id                       346465 non-null  int64         
 1   category                 346465 non-null  int64         
 2   first_release_date       346464 non-null  datetime64[ns]
 3   name                     346465 non-null  object        
 4   release_dates            342352 non-null  datetime64[ns]
 5   rating                   99063 non-null   float64       
 6   rating_count             99063 non-null   float64       
 7   total_rating             114652 non-null  float64       
 8   total_rating_count       114652 non-null  float64       
 9   aggregated_rating        65227 non-null   float64       
 10  aggregated_rating_count  65227 non-null   float64       
dtypes: datetime64[ns](2), float64(6), int64(2), object(1)
memory usage: 29.1+ MB


In [ ]:
#the drop duplicates thing from earlier
cleaned_df.drop_duplicates(subset=['id'], inplace=True)

In [ ]:
# Print the column information for the dataframe
cleaned_df.head(50)

,id,category,first_release_date,name,release_dates,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
0,131913,0,2016-09-21,Maji Kyun! Renaissance,2016-09-21,NaN,NaN,NaN,NaN,NaN,NaN
1,88308,0,2016-11-09,Hey Duggee: The Big Outdoor App,2016-11-09,NaN,NaN,NaN,NaN,NaN,NaN
2,63308,0,2013-07-03,Soldier Front 2,2013-07-03,NaN,NaN,NaN,NaN,NaN,NaN
3,231577,0,2023-02-23,Blood Bowl 3: Black Orcs Edition,2023-02-23,NaN,NaN,NaN,NaN,NaN,NaN
5,119025,0,2019-06-08,Pet Puzzle,2019-06-08,NaN,NaN,NaN,NaN,NaN,NaN
6,213046,3,2022-08-08,The Ultimate FMV Bundle 2,2022-08-08,NaN,NaN,NaN,NaN,NaN,NaN
7,41342,0,1988-03-04,19: Neunzehn,1988-03-04,NaN,NaN,NaN,NaN,NaN,NaN
8,108308,0,2018-08-18,Road Doom,2018-08-18,NaN,NaN,NaN,NaN,NaN,NaN
9,203546,0,2022-06-07,Meat & Greed,2022-06-07,NaN,NaN,NaN,NaN,NaN,NaN
10,99620,0,2018-04-30,Shadow Wolf Mysteries: Cursed Wedding - Collec...,2018-04-30,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#testing to see if the data was cleaned correctly
BG3_3 = cleaned_df.loc[cleaned_df['name'] == "Baldur's Gate 3"]
BG3_3

,id,category,first_release_date,name,release_dates,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
77660,119171,0,2020-10-06,Baldur's Gate 3,2023-09-06,95.203873,379.0,94.851937,396.0,94.5,17.0


In [ ]:
#output the cleaned data to a csv file
cleaned_df.to_csv('cleaned_video_games.csv', index=False)